In [ ]:
import os 
import time
import sys 
import json
from argparse import Namespace

from torch.utils.data import DataLoader
from torchvision.transforms import v2

sys.path.append("./models/ExpansionNet_v2")
from projectCode.ModelLoader import loadModel
from projectCode.CocoDataset import CocoDataset
from projectCode.Perturbator import *
from projectCode.Evaluator import *

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
    
print(device)
device = "cpu"

In [ ]:
ann_file = "./annotations/annotations/captions_val2017.json"
num_examples = 256
seed = 42

ds = CocoDataset(ann_file, num_examples, img_dir = "imgs",seed = seed)

In [ ]:
model_params={"device":device,"tokens_path":'./models/ExpansionNet_v2/demo_material/demo_coco_tokens.pickle'}
model = loadModel("ExpansionNet",model_path="./models/ExpansionNet_v2/rf_model.pth",**model_params)
#model = loadModel("BLIP",model_path = None, device=device)

In [ ]:
"""
corruption_types = [Perturbation(), 
                    Blur(), 
                    Noise(noise_level = 0.12),
                    Noise(noise_type = "Uniform",noise_level = 0.33),
                    Noise(noise_type ="Laplace",noise_level = 0.1), 
                    Patch(patch_size = (50,50))]
"""
corruption_types = [Mirror(axis=[-2,-1]),
                    Mirror(axis=[-1]),
                    Perspective(),
                    RandomRotation(),
                    #Grayscale(),
                    ColorJitter()]
evaluator = Evaluator(model,ds,corruption_types,"METEOR",batch_size=4, seed=seed, device=device)

In [ ]:
"""
import matplotlib.pyplot as plt
imgs = [ds[i][0] for i in range(3)]
b = torch.stack(imgs)
print(b.shape)

transform = T.ToPILImage()
for c in corruption_types:
    img_c = c.apply(b[0])
    img_pil = transform(img_c)
    plt.imshow(img_pil)  # Use cmap="gray" for grayscale images
    plt.axis("off")  # Hide axis
    plt.show()
"""

In [ ]:
scores = evaluator.evaluate_dataset()

In [ ]:
scores_tensor = torch.tensor(scores)

"""
result = {
    "seed": seed,
    "corruption_types":['id','blur','normal noise','uniform noise','laplace noise','patch'],
    "scores": {
        "id": scores_tensor[0].mean().item(),
        "blur": scores_tensor[1].mean().item(),
        "normal noise": scores_tensor[2].mean().item(),
        "uniform noise": scores_tensor[3].mean().item(),
        "laplace noise": scores_tensor[4].mean().item(),
        "patch": scores_tensor[5].mean().item(),
    }
}
"""

result = {
    "seed": seed,
    "corruption_types":['mirror_lr','mirror_ud','random_perspective','random_rotation','random_color'],
    "scores": {
        "mirror_lr": scores_tensor[0].mean().item(),
        "mirror_ud": scores_tensor[1].mean().item(),
        "random_perspective": scores_tensor[2].mean().item(),
        "random_rotation": scores_tensor[3].mean().item(),
        "random_color": scores_tensor[4].mean().item(),
    }
}

print(result)

In [ ]:
file_name = "ExpansionNet_256_BLEU_Mirror_Perspective_Rotation_Color.json"

with open(file_name, 'w') as json_file:
    json.dump(result, json_file)

print(f"Results have been saved to {file_name}")